# Guided Hunting - Azure Resource Explorer

In [10]:
from pathlib import Path
import os
import sys
import warnings
from IPython.display import display, HTML, Markdown
from msticpy import nbwidgets

display(HTML("Checking for msticpy update."))
#%pip install --upgrade msticpy

import msticpy as mp

mp.init_notebook(namespace=globals())

In [1]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "4e3edda0-4a9b-41ee-8197-23223ec3de4b",
    "id": "553563e5-5912-4c7e-be77-a1927bc0286d",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantDefaultDomain": "rnarldosj0921gmail.onmicrosoft.com",
    "tenantDisplayName": "Default Directory",
    "tenantId": "4e3edda0-4a9b-41ee-8197-23223ec3de4b",
    "user": {
      "name": "rnarldo.sj0921@gmail.com",
      "type": "user"
    }
  }
]


In [ ]:
# connect to Azure workspace
ws_config = mp.WorkspaceConfig()
if not ws_config.config_loaded:
    ws_config.prompt_for_ws()

In [6]:
# connect to resourcegraph and loganalytics
qp_RG = mp.QueryProvider("ResourceGraph")
qp_RG.connect(ws_config)

qp_LA = mp.QueryProvider("LogAnalytics")
qp_LA.connect(ws_config)

2025-08-16 21:33:41,950: WARNING - ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (msal_managed_identity_client#191)
2025-08-16 21:33:41,953: WARNING - ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (decorators#51)
2025-08-16 21:33:46,808: WARNING - ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (msal_managed_identity_client#191)
2025-08-16 21:33:46,811: WARNING - ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (decorators#51)


Connected


2025-08-16 21:33:56,353: WARNING - ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (msal_managed_identity_client#191)
2025-08-16 21:33:56,356: WARNING - ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (decorators#51)


Unable to sign-in with Managed Instance credentials.


2025-08-16 21:34:01,234: WARNING - ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (msal_managed_identity_client#191)
2025-08-16 21:34:01,237: WARNING - ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (decorators#51)


Unable to sign-in with Managed Instance credentials.


2025-08-16 21:34:07,899: WARNING - ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (msal_managed_identity_client#191)
2025-08-16 21:34:07,903: WARNING - ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (decorators#51)


Unable to sign-in with Managed Instance credentials.


2025-08-16 21:34:12,753: WARNING - ImdsCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (msal_managed_identity_client#191)
2025-08-16 21:34:12,755: WARNING - ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint. (decorators#51)


Unable to sign-in with Managed Instance credentials.
connected


In [11]:
q_times = nbwidgets.QueryTime(units='day', max_before=20, before=5, max_after=1)
q_times.display()

In [12]:
selected_resourceName = widgets.Text(
    placeholder = 'insert resource ID',
    description = 'Resource ID:',
    disabled = False
)

display(selected_resourceName)

Text(value='', description='Resource ID:', placeholder='insert resource ID')

In [15]:
alert_query = f"""
SecurityAlert
| where TimeGenerated >= datetime("{q_times.start}")
| where TimeGenerated <= datetime("{q_times.end}")
| where isnotempty(ResourceId)
| extend json_extendProp = parse_json(ExtendedProperties)
| extend UserName = json_extendProp['User Name'], ServiceId = json_extendProp['ServiceId'], WdatpTenantId = json_extendProp['WdatpTenantId'], FileName = json_extendProp['File Name'], resourceType = json_extendProp['resourceType'], AttackerSourceIP = json_extendProp['Attacker source IP'], numFailedAuthAttemptsToHost = json_extendProp['Number of failed authentication attempts to host'], numExistingAccountsUsedBySource = json_extendProp['Number of existing accounts used by source to sign in'], numNonExistentAccountsUsedBySource = json_extendProp['Number of nonexistent accounts used by source to sign in'], topAccountsWithFailedSignInAttempts = json_extendProp['Top accounts with failed sign in attempts (count)'], RDPSessionInitiated = json_extendProp['Was RDP session initiated'], attackerSourceComputerName = json_extendProp['Attacker source computer name']
| project-away json_extendProp
"""
alert_df = qp_LA.exec_query(alert_query)

sum_alert_query = f"""
SecurityAlert
| where TimeGenerated >= datetime("{q_times.start}")
| where TimeGenerated <= datetime("{q_times.end}")
| where isnotempty(ResourceId)
| extend json_extendProp = parse_json(ExtendedProperties)
| extend UserName = json_extendProp['User Name'], ServiceId = json_extendProp['ServiceId'], WdatpTenantId = json_extendProp['WdatpTenantId'], FileName = json_extendProp['File Name'], resourceType = json_extendProp['resourceType'], AttackerSourceIP = json_extendProp['Attacker source IP'], numFailedAuthAttemptsToHost = json_extendProp['Number of failed authentication attempts to host'], numExistingAccountsUsedBySource = json_extendProp['Number of existing accounts used by source to sign in'], numNonExistentAccountsUsedBySource = json_extendProp['Number of nonexistent accounts used by source to sign in'], topAccountsWithFailedSignInAttempts = json_extendProp['Top accounts with failed sign in attempts (count)'], RDPSessionInitiated = json_extendProp['Was RDP session initiated'], attackerSourceComputerName = json_extendProp['Attacker source computer name']
| project-away json_extendProp
| summarize count() by AlertName, AlertSeverity, CompromisedEntity, tostring(resourceType)
| sort by count_
"""

sum_alert_df = qp_LA.exec_query(sum_alert_query)
display(sum_alert_df)

,AlertName,AlertSeverity,CompromisedEntity,resourceType,count_


In [16]:
resource_types = [i if i else "N/A" for i in alert_df.resourceType]
resources = set(zip(alert_df.CompromisedEntity, resource_types))
resources = [i for i in resources if i[0]]
resources = [str(i).replace('(','').replace(')','').replace("'", '') for i in resources]
resource_dropdown = widgets.Dropdown(options = resources, description='Resource:')
display(resource_dropdown)

Dropdown(description='Resource:', options=(), value=None)

## View Resource Graph

In [17]:
import networkx as nx
from bokeh.io import output_notebook, show, save
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool, MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool, ColumnDataSource, LabelSet)
from bokeh.plotting import figure
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8, Blues256
from bokeh.transform import linear_cmap, factor_cmap
from networkx.algorithms import community
from ipywidgets import interact, interactive, fixed, interact_manual
from bokeh.io import push_notebook, show, output_notebook

output_notebook()

Loading BokehJS ...

In [18]:
# Query ResourceGraph for resource info
if selected_resourceName.value == '':
    print("SELECTED: ", resource_dropdown.value.split(',')[0])
    rg_query = f"""
    Resources
    | where name == "{resource_dropdown.value.split(',')[0]}"
    """
else:
    print("SELECTED: ", selected_resourceName.value)
    rg_query = f"""
    Resources
    | where name == "{selected_resourceName.value}"
    """

rg_df = qp_RG.exec_query(rg_query)
display(pd.DataFrame(rg_df.iloc[0].T))

try:
    resource_id_list = [rg_df['id'][0]]
    rg = rg_df['resourceGroup'][0]
    print("Resource found!")

    related_rg_query = f"""
    Resources
    | where resourceGroup == "{rg}"
    """

    related_rg_df = qp_RG.exec_query(related_rg_query)
    resource_id_list.extend(list(related_rg_df['id']))
    related_rg_df['managedByVal'] = related_rg_df['managedBy'].str.split('/').str[-1]

except:
    print("No results for that resource. Please select a different resource above.")

AttributeError: 'NoneType' object has no attribute 'split'